In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

# Define StockLSTM Model
class StockLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_hidden_state = lstm_out[:, -1, :]
        output = self.fc(last_hidden_state)
        return output

# Model Hyperparameters
input_size = 3  # Features: High, Low, Close
hidden_size = 64
num_layers = 2
output_size = 3  # Predict High, Low, and Close
sequence_length = 10  # Sequence length

# Create Model
def createModel():
    model = StockLSTM(input_size, hidden_size, num_layers, output_size)
    return model

# Denormalize Function
def denormalize(scaler, data):
    return scaler.inverse_transform(data)

# Train Model and Calculate Errors
def trainModel(model, dataloader):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    epochs = 15
    mse_losses = []
    mean_errors = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        epoch_error = 0.0

        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            predictions = model(X_batch)

            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()

            # Calculate MSE and ME
            epoch_loss += loss.item()
            epoch_error += torch.mean(torch.abs(predictions - y_batch)).item()

        mse_losses.append(epoch_loss / len(dataloader))
        mean_errors.append(epoch_error / len(dataloader))

        print(f"Epoch {epoch+1}, MSE: {mse_losses[-1]:.6f}, Mean Error: {mean_errors[-1]:.6f}")

    return mse_losses, mean_errors

# Prepare DataLoader
def createDataLoader(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length, :])
        y.append(data[i+sequence_length, :])
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    dataset = torch.utils.data.TensorDataset(X, y)
    return torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Prediction Function
def predictStockPrices(date_input, ticker):
    selected_date = datetime.strptime(date_input, "%Y-%m-%d").date()

    # Fetch data for training
    start_date = selected_date - timedelta(days=365)
    end_date = selected_date
    df = yf.download(ticker, start=start_date, end=end_date)

    # Preprocess data
    df = df[['High', 'Low', 'Close']].dropna().values
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)

    # Prepare model and data loader
    model = createModel()
    dataloader = createDataLoader(scaled_data, sequence_length)
    mse_losses, mean_errors = trainModel(model, dataloader)

    # Predict next 6 days
    predictions = []
    input_sequence = scaled_data[-sequence_length:, :]  # Last sequence for prediction
    model.eval()
    with torch.no_grad():
        for _ in range(6):
            input_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0)
            next_day_prediction = model(input_tensor).numpy()[0]
            predictions.append(next_day_prediction)
            input_sequence = np.vstack((input_sequence[1:], next_day_prediction))

    # Denormalize predictions
    predictions = np.array(predictions)
    denorm_predictions = denormalize(scaler, predictions)

    return denorm_predictions, mse_losses, mean_errors

# Main Execution
if __name__ == "__main__":
    date_input = "2024-12-12"  # Example date
    ticker = "NVDA"  # NVIDIA Stock

    # Run predictions and evaluate
    predictions, mse_losses, mean_errors = predictStockPrices(date_input, ticker)

    print("\nFinal Predictions for Next 6 Days:")
    for i, day in enumerate(predictions):
        print(f"Day {i+1}: High={day[0]:.2f}, Low={day[1]:.2f}, Close={day[2]:.2f}")

    print("\nMSE and Mean Error (by epoch):")
    for epoch, (mse, me) in enumerate(zip(mse_losses, mean_errors)):
        print(f"Epoch {epoch+1}: MSE={mse:.6f}, Mean Error={me:.6f}")

[*********************100%***********************]  1 of 1 completed


Epoch 1, MSE: 0.309620, Mean Error: 0.490262
Epoch 2, MSE: 0.140530, Mean Error: 0.309284
Epoch 3, MSE: 0.059293, Mean Error: 0.196727
Epoch 4, MSE: 0.041224, Mean Error: 0.174217
Epoch 5, MSE: 0.030570, Mean Error: 0.145249
Epoch 6, MSE: 0.019718, Mean Error: 0.112913
Epoch 7, MSE: 0.012139, Mean Error: 0.090129
Epoch 8, MSE: 0.007363, Mean Error: 0.067676
Epoch 9, MSE: 0.006274, Mean Error: 0.061639
Epoch 10, MSE: 0.005580, Mean Error: 0.059059
Epoch 11, MSE: 0.005183, Mean Error: 0.057123
Epoch 12, MSE: 0.004824, Mean Error: 0.054688
Epoch 13, MSE: 0.004629, Mean Error: 0.052932
Epoch 14, MSE: 0.004534, Mean Error: 0.052391
Epoch 15, MSE: 0.004179, Mean Error: 0.050897

Final Predictions for Next 6 Days:
Day 1: High=142.94, Low=137.69, Close=139.35
Day 2: High=143.04, Low=137.77, Close=139.48
Day 3: High=143.03, Low=137.76, Close=139.48
Day 4: High=142.98, Low=137.72, Close=139.44
Day 5: High=142.92, Low=137.67, Close=139.38
Day 6: High=142.73, Low=137.49, Close=139.18

MSE and Mean